In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = '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',
    model_name = "llama-3.3-70b-versatile",
)
response = llm.invoke(" The first person to land on moon was ..")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
loader = WebBaseLoader("https://careers.nike.com/3d-footwear-designer-ii/job/R-60083")
page_data = loader.load().pop().page_content
print(page_data)

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': '3D Footwear Designer II',
 'experience': '2-3 years of relevant work experience',
 'skills': ['3D Design techniques',
  'Emerging tools and technologies',
  'Knowledge of product creation',
  'Strong aesthetic sensibility',
  'Technical skills',
  'Proficiency in 3D Design, polygonal and surface modeling',
  'Digital design applications (i.e. Maya, Blender, Rhino, ZBrush, Blender, Photoshop, and Illustrator)',
  'AI (Artificial Intelligence) tools',
  'Footwear product experience'],
 'description': 'As a 3D Footwear Designer, you will be working under the Digital Product Creation Organization while focusing on 3D Design for Global Footwear. You will also collaborate with other members of the design team including product, materials, color, computational designers as well as Development and Marketing as part of a creative collective on each project.'}

In [9]:
type(json_res)

dict

In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [ ]:
!pip install --upgrade onnxruntime

In [ ]:

pip install onnxruntime==1.14.0

In [ ]:
import onnxruntime
print(onnxruntime.__version__)

In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links